### Evaluation of Tree Instance Segmentation Accuracy

In [ ]:
import os

import numpy as np
import pandas as pd

from pointtree.instance_segmentation import MultiStageAlgorithm
from pointtree.evaluation import match_instances, instance_segmentation_metrics
from pointtree.io import PointCloudReader, PointCloudWriter

#### Point cloud files

In [ ]:
base_dir = '<insert path>'

datasets = {
    'TreeML': {
        '2023-01-09_tum_campus': {
            'file_path': '2023-01-09_tum_campus.laz',
            'street': '2023-01-09\_tum\_campus',
            'part': '',
            'subset': 'test',
            'short': 'TTC'
        },
        '2023-01-12_57': {
            'file_path': '2023-01-12_57.laz',
            'street': '2023-01-12\_57',
            'part': '',
            'subset': 'test',
            'short': 'T12\_57'
        },
        '2023-01-12_58': {
            'file_path': '2023-01-12_58.laz',
            'street': '2023-01-12\_58',
            'part': '',
            'subset': 'test',
            'short': 'T12\_58'
        },
        '2023-01-16_12': {
            'file_path': '2023-01-16_12.laz',
            'street': '2023-01-16\_12',
            'part': '',
            'subset': 'test',
            'short': 'T16\_12'
        },
        '2023-01-16_43': {
            'file_path': '2023-01-16_43.laz',
            'street': '2023-01-16\_43',
            'part': '',
            'subset': 'test',
            'short': 'T16\_43'
        },
    },
    'Essen': {
        'altendorfer_part_1': {
            'file_path': 'Altendorfer_p1_min_1.laz',
            'street': 'Altendorfer Straße',
            'part': 'part 1',
            'subset': 'val',
            'short': 'EAD1'
        },
        'altendorfer_part_2': {
            'file_path': 'Altendorfer_p2_min_1.laz',
            'street': 'Altendorfer Straße',
            'part': 'part 2',
            'subset': 'val',
            'short': 'EAD2'
        },
        'altenessener_part_4': {
            'file_path': 'Essen3_p2_min_1.laz',
            'street': 'Altenessener Straße',
            'part': 'part 4',
            'subset': 'val',
            'short': 'EAE4'
        },
        'altenessener_part_5': {
            'file_path': 'Essen3_p3_min_1.laz',
            'street': 'Altenessener Straße',
            'part': 'part 5',
            'subset': 'test',
            'short': 'EAE5'
        }
    },
    'Hamburg': {
        'armgart_straße_part_1': {
            'file_path': '000274_v2_min_1.laz',
            'street': 'Armgartstraße',
            'part': 'part 1',
            'subset': 'val',
            'short': 'HAG1'
        },
        'armgart_straße_part_2': {
            'file_path': '000275_000276_min_1.laz',
            'street': 'Armgartstraße',
            'part': 'part 2',
            'subset': 'test',
            'short': 'HAG2'
        }
    }
}

#### Obtain tree instance segmentation for each point cloud file

In [ ]:
reader = PointCloudReader()
writer = PointCloudWriter()

algorithm_variants = [
    ("w_c", "watershed_crown_top_positions", False),
    ("w_tc", "watershed_matched_tree_positions", False),
    ("w_v", "watershed_matched_tree_positions", True),
    ("w_rg", "full", True),
]

for dataset in datasets:
    os.makedirs(os.path.join(base_dir, 'Data', dataset), exist_ok=True)
    for file_id, file_infos in datasets[dataset].items():
        for algorithm_variant, algorithm_name, correct_watershed in algorithm_variants:
            for (experiment_label, classification_column) in [
                ('gt', 'classification_target'),
                ('dl', 'classification_prediction')
            ]:
                if experiment_label == "gt" and dataset == "TreeML":
                    continue

                print("Process", file_id, algorithm_variant, experiment_label)
                
                instance_seg_folder = os.path.join(base_dir, 'Data', dataset, '5_instance_segmentation')
                os.makedirs(instance_seg_folder, exist_ok=True)
                visualization_folder = os.path.join(base_dir, 'Data', dataset, '6_instance_segmentation_visualizations',
                                                    file_id, experiment_label)
                os.makedirs(visualization_folder, exist_ok=True)
                metrics_folder = os.path.join(base_dir, 'Metrics', dataset, file_id, algorithm_variant)
                os.makedirs(metrics_folder, exist_ok=True)

                read_columns = ["x", "y", "z", "instance_id", "classification_target", "classification_prediction"]
                point_cloud_io_data = reader.read(os.path.join(base_dir, 'Data', dataset,
                                                               '3_semantic_segmentation_processed',
                                                               file_infos["file_path"]), columns=read_columns)
                point_cloud = point_cloud_io_data.data

                algorithm = MultiStageAlgorithm(trunk_class_id=1, crown_class_id=2, branch_class_id=3,
                                                algorithm=algorithm_name,
                                                correct_watershed=correct_watershed,
                                                downsampling_voxel_size=0.03,
                                                visualization_folder=visualization_folder)

                point_cloud["instance_id_predicted"] = algorithm(point_cloud,
                                                                 f'{dataset}_{file_id}_{algorithm_variant}_{experiment_label}',
                                                                 semantic_segmentation_column=classification_column)
                
                file_name = f'{".".join(file_infos["file_path"].split(".")[:-1])}_{algorithm_variant}_{experiment_label}.laz'
                point_cloud_io_data.data = point_cloud
                columns_to_keep = ["x", "y", "z", "instance_id", "instance_id_predicted", "classification_target", "classification_prediction"]
                writer.write(point_cloud_io_data, os.path.join(instance_seg_folder, file_name), columns=columns_to_keep)

                runtime_stats = algorithm.runtime_stats()
                runtime_stats["Dataset"] = dataset
                runtime_stats["FileID"] = file_id
                runtime_stats["Street"] = file_infos['street']
                runtime_stats["Part"] = file_infos['part']
                runtime_stats["SemanticSegmentation"] = experiment_label
                file_name = f'{".".join(file_infos["file_path"].split(".")[:-1])}_instance_segmentation_{algorithm_variant}_{experiment_label}_runtime.csv'
                runtime_stats.to_csv(os.path.join(metrics_folder, file_name), index=False)

#### Calculate tree instance segmentation metrics for each point cloud file

In [ ]:
reader = PointCloudReader()
writer = PointCloudWriter()

algorithm_variants = [
    ("w_c", "watershed_crown_top_positions", False),
    ("w_tc", "watershed_matched_tree_positions", False),
    ("w_v", "watershed_matched_tree_positions", True),
    ("w_rg", "full", True),
]

for dataset in datasets:
    os.makedirs(os.path.join(base_dir, 'Data', dataset), exist_ok=True)
    for file_id, file_infos in datasets[dataset].items():
        for algorithm_variant, algorithm_name, correct_watershed in algorithm_variants:
            for (experiment_label, classification_column) in [
                ('gt', 'classification_target'),
                ('dl', 'classification_prediction')
            ]:
                if experiment_label == "gt" and dataset == "TreeML":
                    continue

                print("Process", file_id, algorithm_variant, experiment_label)
                
                instance_seg_folder = os.path.join(base_dir, 'Data', dataset, '5_instance_segmentation')
                metrics_folder = os.path.join(base_dir, 'Metrics', dataset, file_id, algorithm_variant)
                os.makedirs(metrics_folder, exist_ok=True)
                
                file_name = f'{".".join(file_infos["file_path"].split(".")[:-1])}_{algorithm_variant}_{experiment_label}.laz'
                point_cloud = reader.read(os.path.join(instance_seg_folder, file_name)).data
                point_cloud = point_cloud[np.logical_or(point_cloud["classification_target"].isin([1, 2, 3]),
                                                        point_cloud["classification_prediction"].isin([1, 2, 3]))]

                matched_target_ids, matched_predicted_ids, instance_metrics = match_instances(point_cloud["instance_id"].to_numpy(),
                                                                                  point_cloud["instance_id_predicted"].to_numpy())

                metrics = instance_segmentation_metrics(matched_target_ids, matched_predicted_ids, instance_metrics)

                metrics["Dataset"] = dataset
                metrics["SemanticSegmentation"] = experiment_label.upper()
                metrics["FileID"] = file_id
                metrics["Street"] = file_infos['street']
                metrics["Part"] = file_infos['part']
                metrics["Short"] = file_infos['short']

                metrics = pd.DataFrame([metrics])
            
                file_name = ".".join(file_infos['file_path'].split(".")[:-1]) + f"_instance_segmentation_{algorithm_variant}_{experiment_label}.csv"
                metrics.to_csv(os.path.join(metrics_folder, file_name), index=False)

                file_name = ".".join(file_infos['file_path'].split(".")[:-1]) + f"_instance_segmentation_ious_{algorithm_variant}_{experiment_label}.csv"
                instance_metrics["SemanticSegmentation"] = experiment_label.upper()
                instance_metrics.to_csv(os.path.join(metrics_folder, file_name), index=False)


####  Aggregate metrics for all point cloud files

In [ ]:
def to_camel_case(snake_str):
    return "".join(x.capitalize() for x in snake_str.lower().split("_"))

In [ ]:
output_decimals = 5
output_decimnals_runtime = 2

metrics = []
runtime_results = []

add_runtime_description = True
for algorithm_variant in ["w_rg", "w_c", "w_tc", "w_v"]:
    for dataset in datasets:
        dataset_metrics = []
        dataset_instance_metrics = []
        for file_id, file_infos in datasets[dataset].items():
            for experiment_label in ["gt", "dl"]:
                metrics_folder = os.path.join(base_dir, 'Metrics', dataset, file_id, algorithm_variant)

                file_name = f'{".".join(file_infos["file_path"].split(".")[:-1])}_instance_segmentation_{algorithm_variant}_{experiment_label}_runtime.csv'
                runtime_metrics_file = os.path.join(metrics_folder, file_name)
                if os.path.exists(runtime_metrics_file) and algorithm_variant == "w_rg":
                    runtime_metrics = pd.read_csv(runtime_metrics_file)
                    aggregated_metrics = [{
                        "Description": "Detection of trunk positions",
                        "Runtime": runtime_metrics[runtime_metrics["Description"].isin(["Clustering of trunk points", "Filtering of trunk clusters", "Computation of trunk positions"])]["Runtime"].sum()
                    },
                    {
                        "Description": "Construction of canopy height model",
                        "Runtime": runtime_metrics[runtime_metrics["Description"].isin(["Height map computation"])]["Runtime"].sum()
                    },
                    {
                        "Description": "Region growing segmentation",
                        "Runtime": runtime_metrics[runtime_metrics["Description"].isin(["Seed Mask computation", "Computation of crown distance fields", "Region growing"])]["Runtime"].sum()
                    },
                    {
                        "Description": "Correction of Watershed segmentation",
                        "Runtime": runtime_metrics[runtime_metrics["Description"].isin(["Voronoi correction"])]["Runtime"].sum()
                    }
                    ]
                    runtime_metrics = runtime_metrics[runtime_metrics["Description"].isin(["Voxel-based downsampling", "Detection of crown top positions", "Position matching", "Watershed segmentation", "Upsampling of labels", "Total"])]
                    runtime_metrics = pd.concat([runtime_metrics, pd.DataFrame(aggregated_metrics)])
                    runtime_column = to_camel_case(f"{dataset}_{file_id}_{algorithm_variant}_{experiment_label}")
                    runtime_column = runtime_column.replace("1", "One").replace("2", "Two")
                    runtime_metrics[runtime_column] = runtime_metrics["Runtime"]
                    if add_runtime_description:
                        runtime_results.append(runtime_metrics["Description"])
                        add_runtime_description = False
                    runtime_results.append(runtime_metrics[runtime_column])

                file_name = ".".join(file_infos['file_path'].split(".")[:-1]) + f"_instance_segmentation_{algorithm_variant}_{experiment_label}.csv"
                instance_segmentation_metrics_file = os.path.join(metrics_folder, file_name)

                if os.path.exists(instance_segmentation_metrics_file):
                    current_metrics = pd.read_csv(instance_segmentation_metrics_file)
                    current_metrics["Algorithm"] = algorithm_variant.replace("_", "-").upper()
                    metrics.append(current_metrics)
                    dataset_metrics.append(current_metrics)

                file_name = ".".join(file_infos['file_path'].split(".")[:-1]) + f"_instance_segmentation_ious_{algorithm_variant}_{experiment_label}.csv"
                instance_metrics_file = os.path.join(metrics_folder, file_name)

                if os.path.exists(instance_metrics_file):
                    current_metrics = pd.read_csv(instance_metrics_file)
                    dataset_instance_metrics.append(current_metrics)

        if len(dataset_metrics) > 0:
            dataset_metrics = pd.concat(dataset_metrics)
            dataset_instance_metrics = pd.concat(dataset_instance_metrics)
            for experiment_label in ["GT", "DL"]:
                if experiment_label == "GT" and dataset == "TreeML":
                    continue
                total_tp = dataset_metrics[dataset_metrics["SemanticSegmentation"] == experiment_label]["tp"].sum()
                total_fp = dataset_metrics[dataset_metrics["SemanticSegmentation"] == experiment_label]["fp"].sum()
                total_fn = dataset_metrics[dataset_metrics["SemanticSegmentation"] == experiment_label]["fn"].sum()
                total_f1_score = 2 * total_tp / (2 * total_tp + total_fp + total_fn)
                total_m_iou = dataset_instance_metrics[dataset_instance_metrics["SemanticSegmentation"] == experiment_label]["IoU"].mean()
                total_m_precision = dataset_instance_metrics[dataset_instance_metrics["SemanticSegmentation"] == experiment_label]["Precision"].mean()
                total_m_recall = dataset_instance_metrics[dataset_instance_metrics["SemanticSegmentation"] == experiment_label]["Recall"].mean()
                metrics.append(pd.DataFrame([{
                    "Dataset": dataset,
                    "FileID": "Total",
                    "Street": "Total",
                    "Part": "",
                    "Short": f"{dataset[0].upper()}Total",
                    "Algorithm": algorithm_variant.replace("_", "-").upper(),
                    "SemanticSegmentation": experiment_label,
                    "panoptic_quality": total_f1_score * total_m_iou,
                    "tp": total_tp,
                    "fp": total_fp,
                    "fn": total_fn,
                    "detection_f1_score": total_f1_score,
                    "detection_precision": total_tp / (total_tp + total_fp),
                    "detection_recall": total_tp / (total_tp + total_fn),
                    "segmentation_m_iou": total_m_iou,
                    "segmentation_m_precision": total_m_precision,
                    "segmentation_m_recall": total_m_recall,
                }]))

if len(runtime_results) > 0:
    runtime_results = pd.concat(runtime_results, axis=1)
    for column in runtime_results.columns:
        if column != "Description":
            runtime_results[column] = np.round(runtime_results[column].to_numpy(), output_decimnals_runtime)
    sorted_runtime_metrics = ["Voxel-based downsampling", "Detection of trunk positions", "Construction of canopy height model", "Detection of crown top positions", "Position matching", "Watershed segmentation", "Correction of Watershed segmentation", "Region growing segmentation", "Upsampling of labels", "Total"]
    runtime_results["Description"] = runtime_results["Description"].astype("category")
    runtime_results["Description"] = runtime_results["Description"].cat.set_categories(sorted_runtime_metrics)
    runtime_results = runtime_results.sort_values(["Description"])
    runtime_results.to_csv(os.path.join(base_dir, 'Metrics', 'runtime.csv'), index=False)

if len(metrics) > 0:
    metrics_df = pd.concat(metrics)

    ablation_pq = []
    for file_id in metrics_df["FileID"].unique():
        if file_id == "Total":
            continue
        for semantic_segmentation in ["GT", "DL"]:
            current_metrics = {}
            current_metrics["Dataset"] = metrics_df[metrics_df["FileID"] == file_id]["Dataset"].to_list()[0]
            current_metrics["Street"] = metrics_df[metrics_df["FileID"] == file_id]["Street"].to_list()[0]
            current_metrics["Part"] = metrics_df[metrics_df["FileID"] == file_id]["Part"].to_list()[0]
            current_metrics["SemanticSegmentation"] = semantic_segmentation
            for algorithm_variant in ["W-C", "W-TC", "W-V", "W-RG"]:
                metric = metrics_df[(metrics_df["FileID"] == file_id) & (metrics_df["SemanticSegmentation"] == semantic_segmentation) & (metrics_df["Algorithm"] == algorithm_variant)]
                if len(metric) > 0:
                    assert len(metric) == 1
                    current_metrics[algorithm_variant.replace("-", "")] = np.round(metric["panoptic_quality"].to_list()[0], output_decimals)
            ablation_pq.append(current_metrics)
    ablation_pq = pd.DataFrame(ablation_pq)
    ablation_pq.to_csv(os.path.join(base_dir, 'Metrics', "instance_segmentation_ablation_panoptic_quality.csv"), index=False)

    metric_columns = ["tp", "fp", "fn", "panoptic_quality", "detection_f1_score", "detection_precision", "detection_recall", "segmentation_m_iou", "segmentation_m_precision", "segmentation_m_recall"]
    renamed_metric_columnns = ["DetectionFScore"]
    for metric_column in metric_columns:
        metrics_df[metric_column] = np.round(metrics_df[metric_column].to_numpy(), output_decimals)
        if metric_column != "detection_f1_score":
            renamed_metric_columnns.append(to_camel_case(metric_column))
    metrics_df = metrics_df.rename({column: to_camel_case(column) for column in metric_columns}, axis=1)
    metrics_df = metrics_df.rename({"DetectionF1Score": "DetectionFScore"}, axis=1)
    metrics_df = metrics_df[["Dataset", "Street", "Part", "Short", "Algorithm", "SemanticSegmentation", *renamed_metric_columnns]]
    metrics_df.to_csv(os.path.join(base_dir, 'Metrics', "instance_segmentation_metrics.csv"), index=False)

In [ ]:
selected = metrics_df[(metrics_df["Street"] == "Total") & (metrics_df["Algorithm"]).isin(["W-RG"]) & (metrics_df["SemanticSegmentation"]).isin(["DL", "GT"])]
selected = selected[["Dataset", "Street", "Part", "Algorithm", "DetectionRecall"]]
selected.sort_values(by="Dataset", inplace=True)
selected